# 📊 Análise Exploratória de Dados de da Rede Sonda 🌦️

## 📌 Introdução
Este notebook realiza uma análise exploratória dos dados meteorológicos coletados de diversas estações. O objetivo é entender a estrutura dos dados, avaliar sua qualidade e identificar padrões relevantes.

## 📂 Fonte dos Dados
- Arquivos CSV formatados armazenados no ftp
- Contêm medições de variáveis meteorológicas, solarimétricas e cameras.

## 🔍 Objetivos da Análise
1. **Carregar e explorar os dados**: verificar onde os dados estão armazenados, seu formato e estrutura.
2. **Dimensionamento e variáveis disponíveis**: entender o tamanho dos arquivos, número de registros e colunas.
3. **Análise temporal dos dados disponíveis**: identificar o período coberto e eventuais lacunas temporais.
4. **Visualização da distribuição espacial das estações**: verificar a abrangência geográfica das medições.
5. **Exploração inicial de distribuições**: histogramas e estatísticas básicas das variáveis.
6. **Análise de qualidade dos dados** *(última etapa)*: identificar valores ausentes, inconsistências e flags de qualidade.

### 1. Carregar e Explorar os Dados
Vamos começar listando o tamanho da base de dados que estão no diretório do ftp.

In [1]:
# Diretório onde os arquivos estão localizados
DIRETORIO = '../sonda/dados_formatados/'

In [2]:
# Exibe o tamanho de cada arquivo no diretório ordenado por tamanho de forma decrescente
!du -h --max-depth=1 {DIRETORIO} | sort -rh

11G	../sonda/dados_formatados/
1,2G	../sonda/dados_formatados/BRB
1011M	../sonda/dados_formatados/PTR
989M	../sonda/dados_formatados/FLN
959M	../sonda/dados_formatados/PMA
759M	../sonda/dados_formatados/JOI
722M	../sonda/dados_formatados/CPA
714M	../sonda/dados_formatados/SMS
686M	../sonda/dados_formatados/SLZ
613M	../sonda/dados_formatados/NAT
536M	../sonda/dados_formatados/CGR
464M	../sonda/dados_formatados/SBR
413M	../sonda/dados_formatados/TMA
365M	../sonda/dados_formatados/MCL
349M	../sonda/dados_formatados/ORN
300M	../sonda/dados_formatados/UBE
284M	../sonda/dados_formatados/BJL
175M	../sonda/dados_formatados/TLG
174M	../sonda/dados_formatados/CAI
171M	../sonda/dados_formatados/CTB
55M	../sonda/dados_formatados/CBA
196K	../sonda/dados_formatados/TRI
196K	../sonda/dados_formatados/SPK
196K	../sonda/dados_formatados/SCR
196K	../sonda/dados_formatados/RLM
196K	../sonda/dados_formatados/OPO
196K	../sonda/dados_formatados/MDS
196K	../sonda/dados_formatados/LEB
196K	../sonda/dados_form

Existem 3 tipos de dados:
- Dados Meteorológicos
- Dados Solarimétricos
- Dados Anemometricos

In [3]:
import glob

# listar todos os dados Meteorológicos usando o glob só para o tipo de arquivo .csv
dados_metereologicos = glob.glob(DIRETORIO + "*/Meteorologicos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_metereologicos = [arquivo for arquivo in dados_metereologicos if 'YYYY_MM' not in arquivo]

# listar todos os dados de Solarimétricos usando o glob só para o tipo de arquivo .csv
dados_solarimetricos = glob.glob(DIRETORIO + "*/Solarimetricos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_solarimetricos = [arquivo for arquivo in dados_solarimetricos if 'YYYY_MM' not in arquivo]

# listar todos os dados de Anemometricos usando o glob só para o tipo de arquivo .csv
dados_anemometricos = glob.glob(DIRETORIO + "*/Anemometricos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_anemometricos = [arquivo for arquivo in dados_anemometricos if 'YYYY_MM' not in arquivo]

In [4]:
# Listar a quantidade de arquivos em cada categoria
print(f"Quantidade de arquivos Meteorologicos: {len(dados_metereologicos)}")
print(f"Quantidade de arquivos Solarimetricos: {len(dados_solarimetricos)}")
print(f"Quantidade de arquivos Anemometricos: {len(dados_anemometricos)}")

Quantidade de arquivos Meteorologicos: 1036
Quantidade de arquivos Solarimetricos: 1022
Quantidade de arquivos Anemometricos: 0


In [5]:
# Importar a biblioteca DuckDB para manipulação de dados
import duckdb
import os
import pandas as pd

# Conectar ao banco de dados DuckDB e controla numero de threads
con = duckdb.connect(database=':memory:', read_only=False, config={'threads': 4})

# Remove qualquer arquivo temporário que possa existir
!rm -rf .tmp

In [ ]:
# Função para criar a tabela no banco de dados
def criar_tabela(con, dados, base, saida):
    # Deleta tabela caso exista
    con.execute(f"DROP TABLE IF EXISTS {base}")

    # Verifica se o arquivo existe, caso exista leia o arquivo usando duckdb
    if  os.path.exists(saida):
        # Ler o arquivo parquet parquet usando duckdb
        con.execute(f"CREATE TABLE {base} AS SELECT * FROM read_parquet('{saida}')")
    else:
        # Criar a tabela temporária vazia com base no primeiro arquivo CSV
        query = f"""
        CREATE TABLE {base} AS 
        SELECT * FROM read_csv_auto('{dados[0]}', 
                                    skip=2, 
                                    union_by_name=True, 
                                    all_varchar=True) 
        WHERE 1=0
        """
        con.execute(query)  # Cria a tabela vazia com o esquema correto
        # Inserir os dados em lote sem carregar tudo na memória
        for arquivo in sorted(dados):
            print(f"Inserindo dados do arquivo: {arquivo}", end="\r", flush=True)
            try:
                query = f"""
                INSERT INTO {base} 
                SELECT * FROM read_csv_auto('{arquivo}', skip=2, union_by_name=True, all_varchar=True)
                """
                con.execute(query)
            except Exception as e:
                print(f"⚠️ Erro ao processar o arquivo: {arquivo}")
                print(f"   ➡️ Motivo: {e}")

        # Ler a primeira linha do arquivo CSV para obter os nomes das colunas
        header = pd.read_csv(dados[0], nrows=1)
        column_names = header.columns.tolist()  # Nomes das colunas

        # Renomear as colunas com base nos nomes do arquivo CSV
        for i, col in enumerate(column_names):
            # Formata o nome da coluna conforme a nomenclatura do DuckDB (column01, column02, ...)
            column_name = f"column{str(i).zfill(2)}"  # Preenche com zero à esquerda para 2 dígitos
            # Renomeia a coluna pelo nome correto
            con.execute(f"ALTER TABLE {base} RENAME COLUMN {column_name} TO {col}")

        # Set column timestamp as datetime
        con.execute(f"ALTER TABLE {base} ALTER COLUMN timestamp SET DATA TYPE TIMESTAMP")

        # Seta coluna 1 como STRING
        con.execute(f"ALTER TABLE {base} ALTER COLUMN {column_names[0]} SET DATA TYPE STRING")

        # Todas as demais colunas como FLOAT a partir da segunda coluna
        for i, col in enumerate(column_names[5:], start=5):
            # Formata o dado da coluna caso exista valores com , virgula, substitui por .
            con.execute(f"UPDATE {base} SET {col} = REPLACE({col}, ',', '.')")

            # Formata o dado da coluna caso exista valores com - virgula, substitui por .
            con.execute(f"UPDATE {base} SET {col} = REPLACE({col}, '-', '0')")

            try:
                con.execute(f"ALTER TABLE {base} ALTER COLUMN {col} SET DATA TYPE FLOAT")
            except Exception as e:
                print(f"⚠️ Erro ao processar a coluna: {col}")
                print(f"   ➡️ Motivo: {e}")

        # Seta colunas 2, 3 e 4 como INTEGER
        for i in column_names[2:5]:
            try:
                con.execute(f"ALTER TABLE {base} ALTER COLUMN {i} SET DATA TYPE INTEGER")
            except Exception as e:
                print(f"⚠️ Erro ao processar a coluna: {i}")
                print(f"   ➡️ Motivo: {e}")
        
        # Salvar os dados em Parquet na BASE
        con.execute(f"COPY {base} TO '{saida}' (FORMAT 'parquet')")

In [7]:
# Apontar o caminho das bases de dados
BASE_METEOROLOGICA = '../sonda/dados_meteorologicos.parquet'
BASE_SOLARIMETRICA = '../sonda/dados_solarimetricos.parquet'

In [8]:
# Criar a tabela meteorológica
tabela_meteorologica = 'base_meteorologica'
criar_tabela(con, dados_metereologicos, tabela_meteorologica, BASE_METEOROLOGICA)

In [9]:
# Criar a tabela solarimétrica
tabela_solarimetrica = 'base_solarimetrica'
criar_tabela(con, dados_solarimetricos, tabela_solarimetrica, BASE_SOLARIMETRICA)

⚠️ Erro ao processar a coluna: lw_stddados_formatados/UBE/Solarimetricos/2015/UBE_2015_12_SD_formatado.csv
   ➡️ Motivo: Conversion Error: Could not convert string '-' to FLOAT


In [33]:
# Exibir as primeiras linhas da tabela meteorological
con.execute(f"SELECT * FROM {tabela_meteorologica} LIMIT 5").fetch_df()

,acronym,timestamp,year,day,min,tp_sfc,humid_sfc,press,rain,ws10_avg,ws10_std,wd10_avg,wd10_std
0,BJL,2014-06-01 00:00:00,2014,152,0,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
1,BJL,2014-06-01 00:10:00,2014,152,10,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
2,BJL,2014-06-01 00:20:00,2014,152,20,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
3,BJL,2014-06-01 00:30:00,2014,152,30,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0
4,BJL,2014-06-01 00:40:00,2014,152,40,3333.0,3333.0,3333.0,-5555.0,-5555.0,-5555.0,-5555.0,-5555.0


In [11]:
# Exibir as primeiras linhas da tabela solarimetrica
tabela_solarimetrica = 'base_solarimetrica'
con.execute(f"SELECT * FROM {tabela_solarimetrica} LIMIT 5").fetch_df()

,acronym,timestamp,year,day,min,glo_avg,glo_std,glo_max,glo_min,dif_avg,...,dir_min,lw_avg,lw_std,lw_max,lw_min,temp_glo,temp_dir,temp_dif,temp_dome,temp_case
0,BJL,2014-06-01 00:00:00,2014,152,0,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
1,BJL,2014-06-01 00:01:00,2014,152,1,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
2,BJL,2014-06-01 00:02:00,2014,152,2,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
3,BJL,2014-06-01 00:03:00,2014,152,3,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0
4,BJL,2014-06-01 00:04:00,2014,152,4,3333.0,3333.0,3333.0,3333.0,3333.0,...,3333.0,3333.0,3333.0,3333.0,3333.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Pega apenas colunas de dados meteorológicos
colunas_meteoro = con.execute(f"SELECT * FROM {tabela_meteorologica} LIMIT 1").description
colunas_meteoro = [c[0] for c in colunas_meteoro[5:]]
print(f"Colunas de dados meteorológicos: {colunas_meteoro}")

# Pega apenas colunas de dados solarimétricos
colunas_solar = con.execute(f"SELECT * FROM {tabela_solarimetrica} LIMIT 1").description
colunas_solar = [c[0] for c in colunas_solar[5:]]
print(f"Colunas de dados solarimétricos: {colunas_solar}")

Colunas de dados meteorológicos: ['tp_sfc', 'humid_sfc', 'press', 'rain', 'ws10_avg', 'ws10_std', 'wd10_avg', 'wd10_std']
Colunas de dados solarimétricos: ['glo_avg', 'glo_std', 'glo_max', 'glo_min', 'dif_avg', 'dif_std', 'dif_max', 'dif_min', 'par_avg', 'par_std', 'par_max', 'par_min', 'lux_avg', 'lux_std', 'lux_max', 'lux_min', 'dir_avg', 'dir_std', 'dir_max', 'dir_min', 'lw_avg', 'lw_std', 'lw_max', 'lw_min', 'temp_glo', 'temp_dir', 'temp_dif', 'temp_dome', 'temp_case']


In [48]:
def verificar_dados_invalidos(con, base, colunas): 
    query = f"""
    SELECT acronym, COUNT(*) AS total_dados,
    """
    # Para cada coluna em colunas_meteoro, criamos a parte do "dados_invalidos"
    for i, coluna in enumerate(colunas):
        query += f"""
        SUM(CASE WHEN \"{coluna}\" = 3333.0 THEN 1 ELSE 0 END) AS {coluna}_3333,
        SUM(CASE WHEN \"{coluna}\" = -5555.0 THEN 1 ELSE 0 END) AS {coluna}_minus_5555,
        """
    # Remover a última vírgula da consulta
    query = query.rstrip(",\n")
    # Adiciona a parte do FROM e GROUP BY
    query += f"""
    FROM \"{base}\"
    GROUP BY acronym
    """
    # Executar a query
    df = con.execute(query).fetch_df()
    return df

In [49]:
verificar_dados_invalidos(con, tabela_meteorologica, colunas_meteoro)

,acronym,total_dados,tp_sfc_3333,tp_sfc_minus_5555,humid_sfc_3333,humid_sfc_minus_5555,press_3333,press_minus_5555,rain_3333,rain_minus_5555,ws10_avg_3333,ws10_avg_minus_5555,ws10_std_3333,ws10_std_minus_5555,wd10_avg_3333,wd10_avg_minus_5555,wd10_std_3333,wd10_std_minus_5555
0,CGR,250560,34317.0,0.0,34317.0,0.0,34317.0,0.0,21008.0,118487.0,34317.0,0.0,34317.0,0.0,34317.0,0.0,34320.0,0.0
1,BRB,407520,39203.0,0.0,39203.0,0.0,39203.0,0.0,39200.0,0.0,39203.0,0.0,39203.0,0.0,39203.0,0.0,39203.0,0.0
2,CPA,267264,6167.0,0.0,6167.0,0.0,6167.0,0.0,0.0,267264.0,0.0,267264.0,0.0,267264.0,0.0,267264.0,0.0,267264.0
3,TLG,65664,3627.0,0.0,3627.0,0.0,3627.0,0.0,0.0,65664.0,0.0,65664.0,0.0,65664.0,0.0,65664.0,0.0,65664.0
4,BJL,105120,8839.0,0.0,8839.0,0.0,8839.0,0.0,0.0,105120.0,0.0,105120.0,0.0,105120.0,0.0,105120.0,0.0,105120.0
5,CAI,65808,7397.0,0.0,7397.0,0.0,7397.0,0.0,0.0,65808.0,0.0,65808.0,0.0,65808.0,0.0,65808.0,0.0,65808.0
6,SBR,233037,39426.0,0.0,39433.0,0.0,39420.0,0.0,22629.0,48384.0,43484.0,0.0,43484.0,0.0,39809.0,0.0,39810.0,0.0
7,UBE,127008,19834.0,0.0,19834.0,0.0,19834.0,0.0,0.0,127008.0,0.0,127008.0,0.0,127008.0,0.0,127008.0,0.0,127008.0
8,PMA,465120,55995.0,0.0,55689.0,0.0,55687.0,0.0,55687.0,0.0,55942.0,0.0,55942.0,0.0,55687.0,0.0,55687.0,0.0
9,SLZ,319680,9199.0,0.0,9199.0,0.0,9199.0,0.0,0.0,297648.0,9199.0,0.0,9199.0,0.0,9199.0,0.0,9199.0,0.0


In [50]:
verificar_dados_invalidos(con, tabela_solarimetrica, colunas_solar)

ConversionException: Conversion Error: Could not convert string "640,7" to DECIMAL(5,1)

LINE 4:         SUM(CASE WHEN "glo_avg" = 3333.0 THEN 1 ELSE 0 END) AS glo_avg_3333,
                              ^